## Scraping the ICD website

In [ ]:
import requests
import urllib3
import json
from collections import defaultdict
urllib3.disable_warnings()

In [ ]:
# personal client_id and client_secret for accessing ICD API
id = '0d73f904-277c-46bc-9f26-27ecca8981fb_39a11bc3-ff37-4709-9145-e4ae7bbb19fe'
secret = '263Amruwu0yuyoHNmZ0V5C2GEFEQ7D6vZ5SxKmivDmU='

In [ ]:
token_endpoint = 'https://icdaccessmanagement.who.int/connect/token'
client_id = id
client_secret = secret
scope = 'icdapi_access'
grant_type = 'client_credentials'

In [ ]:
# getting the OAUTH2 token
# setting data to post
payload = {'client_id': client_id,
           'client_secret': client_secret,
           'scope': scope,
           'grant_type': grant_type}

# making request
r = requests.post(token_endpoint, data=payload, verify=False).json()
token = r['access_token']


# accessing ICD API
# link with mental, behavioural or neurodevelopmental disorders
uri = 'https://id.who.int/icd/entity/334423054'

# HTTP header fields to set
headers = {'Authorization':  'Bearer '+ token,
           'Accept': 'application/json',
           'Accept-Language': 'en',
           'API-Version': 'v2'}


In [ ]:
# making request
r = requests.get(uri, headers=headers, verify=False)
# print(r.text)

data = json.loads(r.text)
print(data)

{'@context': 'http://id.who.int/icd/contexts/contextForFoundationEntity.json', '@id': 'http://id.who.int/icd/entity/334423054', 'parent': ['http://id.who.int/icd/entity/455013390'], 'child': ['http://id.who.int/icd/entity/1516623224', 'http://id.who.int/icd/entity/405565289', 'http://id.who.int/icd/entity/486722075', 'http://id.who.int/icd/entity/76398729', 'http://id.who.int/icd/entity/1336943699', 'http://id.who.int/icd/entity/1321276661', 'http://id.who.int/icd/entity/991786158', 'http://id.who.int/icd/entity/108180424', 'http://id.who.int/icd/entity/1412387537', 'http://id.who.int/icd/entity/1884115764', 'http://id.who.int/icd/entity/794195577', 'http://id.who.int/icd/entity/1602669465', 'http://id.who.int/icd/entity/826065555', 'http://id.who.int/icd/entity/310393530', 'http://id.who.int/icd/entity/37291724', 'http://id.who.int/icd/entity/2110604642', 'http://id.who.int/icd/entity/430567349', 'http://id.who.int/icd/entity/213458094', 'http://id.who.int/icd/entity/882114523', 'http

In [ ]:
# getting titles and definitions of every sub-disease
titles = []
definitions = []
icd_defs = defaultdict(list)

for link in data["child"]:
    url = link
    r = requests.get(url, headers=headers, verify=False)
    data = json.loads(r.text)
    # category_title = data["title"]["@value"]
    for link in data["child"]:
        url = link
        r = requests.get(url, headers=headers, verify=False)
        data = json.loads(r.text)
        title = data["title"]["@value"]
        if "definition" in data:
            definition = data["definition"]["@value"]
            definitions.append(definition)
            titles.append(title)
            icd_defs[title].append(definition)
        else:
            continue

In [ ]:
print(len(icd_defs.keys()))
for k, v in icd_defs.items():
    print(k, v)

In [ ]:
# creating a dictionary with needed information to write in a csv file
import csv

field_names= ['No', 'Disease', 'Description']
defs_lst = []
num = 1
for t, d in zip(titles, definitions):
    defs_lst.append({'No': num, 'Disease': t, 'Description': d})
    num +=1

24


In [ ]:
# writing info to a csv file
with open('definitions.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=field_names)
    writer.writeheader()
    writer.writerows(defs_lst)